In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load the data
df = pd.read_csv('Names.csv')

# Инициализация преобразователей
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder()

# Применение LabelEncoder к столбцу "Значение"
df['Значение'] = label_encoder.fit_transform(df['Значение'])

# Применение OneHotEncoder к закодированному столбцу "Значение"
one_hot_encoded = one_hot_encoder.fit_transform(df[['Значение']])

# Добавление векторизированных значений в исходный DataFrame
df_encoded = pd.concat([df, pd.DataFrame(one_hot_encoded.toarray(), columns=one_hot_encoder.get_feature_names_out())], axis=1)




In [2]:
df.head(5)

,Имя,Значение
0,Абга,836
1,Абрагь,128
2,Абрскил,1077
3,Адгур,5115
4,Алхас,3538


In [3]:
# Обратное преобразование закодированных значений обратно в исходные текстовые значения
original_values = label_encoder.inverse_transform(df['Значение'])

# Добавление исходных текстовых значений обратно в DataFrame
df['Original_Value'] = original_values


In [4]:
df.head(5)

,Имя,Значение,Original_Value
0,Абга,836,волк
1,Абрагь,128,арбек
2,Абрскил,1077,герой
3,Адгур,5115,сильный защитник
4,Алхас,3538,особый


In [5]:
duplicates = df['Значение'].value_counts()

print(duplicates)

Значение
5605    213
365     194
2325    143
4502    135
6163    130
       ... 
2709      1
5629      1
818       1
1011      1
401       1
Name: count, Length: 6472, dtype: int64


In [6]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\misha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\misha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize


# Токенизация каждой строки в столбце и создание нового столбца с токенами
df['токены'] = df['Original_Value'].apply(lambda text: word_tokenize(text))

df.head(10)



,Имя,Значение,Original_Value,токены
0,Абга,836,волк,[волк]
1,Абрагь,128,арбек,[арбек]
2,Абрскил,1077,герой,[герой]
3,Адгур,5115,сильный защитник,"[сильный, защитник]"
4,Алхас,3538,особый,[особый]
5,Анос,1513,друг,[друг]
6,Анри,2955,мужественный,[мужественный]
7,Аслан,2476,лев,[лев]
8,Астамур,3086,настоящее железо,"[настоящее, железо]"
9,Ахра,5146,скала,[скала]


In [9]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\misha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import gensim.downloader as api

model = api.load("word2vec-ruscorpora-300")

In [11]:
from gensim.models import Word2Vec
import gensim
from gensim.models import KeyedVectors
from nltk import word_tokenize
import numpy as np

# Преобразование токенов в токенизированные слова
tokenized_text = [word_tokenize(str(text)) for text in df['Original_Value']]

sim_words = model.most_similar('бог_NOUN', topn=5)
sim_words

[('господь_NOUN', 0.8023737668991089),
 ('христос_NOUN', 0.6623279452323914),
 ('всевышний_NOUN', 0.6598852872848511),
 ('божество_NOUN', 0.6551232933998108),
 ('господи_NOUN', 0.6267621517181396)]

In [12]:
import gensim.downloader as api
list(gensim.downloader.info()['models'].keys())

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

In [13]:
v = df['вектор'].iloc[0]
v

KeyError: 'вектор'

In [ ]:
import numpy as np

def k_means(data, k):
    # Инициализация центроид
    centroids = data[np.random.choice(range(len(data)), size=k)]

    while True:
        # Выделение кластеров
        clusters = [[] for i in range(k)]
        for point in data:
            distances = [np.linalg.norm(point - centroid) for centroid in centroids]
            closest_centroid_idx = np.argmin(distances)
            clusters[closest_centroid_idx].append(point)

        # Обновление центроид
        new_centroids = []
        for cluster in clusters:
            if len(cluster) > 0:
                cluster_mean = np.mean(cluster, axis=0)
                new_centroids.append(cluster_mean)

        # Проверка на сходимость
        if np.allclose(centroids, new_centroids):
            break

        centroids = new_centroids

    return centroids

# Пример использования
data = np.array(df['вектор'].tolist(), dtype=float)
k = 1
centroids = k_means(data, k)
print("Центроиды:", centroids)


Центроиды: [array([-1.89634762e-04,  4.40270468e-04,  1.37910399e-04,  5.62945481e-05,
        2.55698031e-05, -8.45498974e-04,  2.16633842e-04,  1.11487649e-03,
       -3.36346503e-04, -3.54324011e-04, -1.29174398e-04, -6.43410997e-04,
       -1.70924495e-04,  2.21697083e-04,  1.52307927e-04, -4.55665537e-04,
        2.73053618e-04, -4.45183983e-04, -1.05147761e-04, -9.39154147e-04,
        2.43280685e-04,  8.66496651e-05,  4.31775424e-04, -2.17258472e-04,
       -1.13268485e-04, -1.98017999e-06, -4.37406524e-04, -5.88555466e-05,
       -3.90110828e-04,  1.31160429e-04,  3.56605856e-04, -3.40682461e-06,
        2.11652233e-04, -5.71329589e-04, -2.53236648e-04,  3.49991860e-04,
        1.63491780e-04, -3.71780947e-04, -2.99055715e-04, -6.66648605e-04,
       -8.93157275e-05, -3.60860731e-04, -3.85029366e-04,  9.48578342e-05,
        2.33459747e-04, -2.98417126e-04, -3.33918355e-04, -7.30270954e-05,
        3.02881000e-04,  4.19240965e-04,  1.05006086e-04, -4.47961198e-04,
       -6.652

In [ ]:
def find_closest_vector(centroid, vectors):
    min_distance = float('inf')
    closest_vector = None
    
    for vector in vectors:
        distance = np.linalg.norm(centroid - vector)
        
        if distance < min_distance:
            min_distance = distance
            closest_vector = vector
    
    return closest_vector

In [ ]:
vectors = data

closest_vector = find_closest_vector(centroids, vectors)
print("Ближайщий вектор:", closest_vector)

Ближайщий вектор: [-1.15074553e-03  1.58622501e-03 -1.98012839e-04 -1.05944707e-03
 -1.82146463e-03 -1.11289947e-03 -2.10964177e-03 -1.77248055e-03
  1.88377490e-03  2.30703297e-03 -5.26114123e-03 -3.64770849e-03
 -1.22325763e-03  8.92060673e-04 -2.95633380e-03 -5.40878760e-04
 -4.15016317e-03 -4.13960379e-04  1.66452197e-03 -7.02757291e-04
 -1.90176129e-03 -5.44882780e-03 -9.60121827e-04  1.28260261e-03
 -8.41276720e-04  1.95933945e-03 -1.99026645e-03 -1.87210787e-03
  1.82763037e-03  3.60042444e-03 -1.00630072e-03 -4.71751671e-04
  3.60191441e-03  2.39741608e-03 -1.46119382e-03 -2.08633797e-03
 -2.13326774e-03  1.16702646e-03  4.55981621e-04 -4.46921991e-04
  3.47536484e-04  1.53957478e-03 -3.88595027e-03  1.24432674e-03
  3.80455322e-03 -1.86853821e-03  3.97730172e-04  2.81343547e-04
 -1.37583175e-03 -2.29409722e-03  2.18316664e-03  2.26574420e-03
  1.58936609e-03 -2.00403044e-03 -2.40874209e-03 -2.34534838e-04
 -8.94967622e-05  3.39622749e-03 -3.85229141e-03 -3.66082450e-03
 -3.040

In [ ]:
word_vector = closest_vector
similar_word = model.wv.most_similar(positive=[word_vector], topn=1)
similar_word

[('лет', 0.4902796149253845)]